In [ ]:
from __future__ import print_function, division   # Python 2/3 compatibility
from skimage import io                            # utilities to read and write images in various formats
import numpy as np  # array manipulation package
import matplotlib.pylab as plt                    # plotting package
%matplotlib inline
plt.rcParams['figure.figsize'] = (7,15)         # set default figure size
plt.rcParams['image.cmap'] = 'gray'               # set default colormap to gray

# Digital Image Processing - Programming Assignment 

The following progamming assignment involves image filtering in the frequency domain. The deadline for returning your work is **14 April 2022 at 23:59. 
Please, follow carefully the submission instructions given in the end of this notebook.** You are encouraged to seek information in other places than the course book and lecture material but remember **list all your sources under references**.

If you experience problems that you cannot solve using the course material or the Python documentation, or have any questions regarding to the programming assignments, please do not hesitate to contact the course assistant by e-mail at the address dip@unioulu.oulu.fi.

**Please, fill in your personal details below.**

# Personal details:

* **Name(s) and student ID(s): Saara Laasonen, 2686040** 
* **Contact information: saara.laasonen@student.oulu.fi** 

# 4. Image transforms : lowpass and highpass filtering in frequency domain

In the following, you will first perform ideal lowpass and highpass filtering on the test image, and later, we will also consider the Gaussian lowpass and highpass filtering. First, read the part concerning image enhancement in frequency domain in the lecture notes or in the course book. Specifically, you should look at the **Chapter-4** (available as a PDF file) in the lecture notes in Moodle.

Now, perform the following operations in the reserved code cells and answer to the questions written in bold into the reserved spaces.


**4.1. Read and display the test image `hplptest.jpg`.**

In [ ]:
# read test image
test = io.imread('hplptest.jpg')

# display test image
plt.imshow(test)
plt.show()

**4.2. Compute the Fourier transform (FT) of the test image and take a look at what the magnitude of the FT looks like.**

Hint: When plotting the FTs, use logarithmic graylevel transformation to make the result more illustrative for human visual system: 

`>>> np.log(np.abs(image_fft)+1)`

In [ ]:
from scipy import fftpack

# compute the FT of the test image using 'fftpack.fft2'
fttest = fftpack.fft2(test)

# translate the origin of the FT (low frequencies) to the center using 'ftpack.fftshift'
centered = fftpack.fftshift(fttest)

# display the magnitude of the uncentered and centered FT using 'imshow'.
fig, ax = plt.subplots(1, 2)
ax[0].imshow(np.log(np.abs(fttest)+1))
ax[0].set_title('uncentered')
ax[1].imshow(np.log(np.abs(centered)+1))
ax[1].set_title('centered')
fig.tight_layout()


**The code for constructing an ideal lowpass filter is given below:**

In [ ]:
# make two frequency matrices, 'f1' and 'f2', as help variables (frequencies from -1 to 1)
n = (500,500)
f1 = ( np.arange(0,n[0])-np.floor(n[0]/2) ) * (2./(n[0]))
f2 = ( np.arange(0,n[1])-np.floor(n[1]/2) ) * (2./(n[1]))
f1, f2 = np.meshgrid(f1, f2)

# make a matrix with absolute values of frequency (“sampled” frequency domain)
D = np.sqrt(f1**2 + f2**2)

# set cut-off frequency D0 to 0.2
D0 = 0.2;

# filter matrix is initialized to ones 
Hlp = np.ones(n)

# set frequencies in filter mask Hlp greater than the cut-off frequency D0 to zero, other elements remain unaltered
Hlp[D>D0] = 0.0

**4.3. Modify the lowpass filter code and construct ideal highpass filter `Hhp` with the same cut-off frequency `D0=0.2` and display both ideal lowpass and highpass filter masks in the same figure.**

In [ ]:
# create ideal highpass filter mask Hhp
Hhp = np.zeros(n)
Hhp[D>D0] = 1.0
# display the filters
fig, ax = plt.subplots(1, 2)
ax[0].imshow(Hlp)
ax[0].set_title('Hlp')
ax[1].imshow(Hhp)
ax[1].set_title('Hhp')
fig.tight_layout()

**4.4. Perform ideal lowpass and highpass filtering in the frequency domain by multiplying the centralized FT of the original image with the `Hlp` and `Hhp` filter masks (element-per-element matrix multiplication) and display the two resulting FTs in the same figure.**

In [ ]:
# apply ideal lowpass and highpass filtering to the test image, i.e. multiply element-wise the fft of the image with the filter masks
hlpfilt = np.multiply(centered,Hlp)
hhpfilt = np.multiply(centered,Hhp)
# display the magnitude of the resulting FTs
fig, ax = plt.subplots(1, 2)
ax[0].imshow(hlpfilt.real)
ax[0].set_title('Hlp filtered')
ax[1].imshow(hhpfilt.real)
ax[1].set_title('Hhp filtered')
fig.tight_layout()

**4.5. Reconstruct the filtered images with `fftpack.ifft2()` and `fftpack.ifftshift()` in reverse order and display the two filtered images using `imshow()` in the same figure.** 

Hint: Due to round-off errors, you have to take the real part of the result of inverse FT before displaying it with `imshow()`. Please note also that the resulting images values beyond the original `uint8` image `[0,255]`, so you need to clip these values using `np.clip()`.

In [ ]:
# reconstruct the filtered images
Hlp_Filt = fftpack.ifft2(fftpack.ifftshift(hlpfilt))
Hhp_Filt = fftpack.ifft2(fftpack.ifftshift(hhpfilt))
# take the 'real' part of the resulting images due to possible round-off errors
Hlp_Filter = np.real(Hlp_Filt)
Hhp_Filter = np.real(Hhp_Filt)
# clip values beyond the uint8 range [0,255] 
Hlp_uint = np.clip(Hlp_Filter, a_min=0, a_max = 255)
Hhp_uint = np.clip(Hhp_Filter, a_min=0, a_max = 255)
# display the original image and its lowpass and highpass filtered images in the same figure
fig, ax = plt.subplots(1, 3)
ax[0].imshow(test)
ax[0].set_title('original')
ax[1].imshow(Hlp_uint)
ax[1].set_title('Hlp filtered')
ax[2].imshow(Hhp_uint)
ax[2].set_title('Hhp filtered')
fig.tight_layout()

When performing ideal lowpass and highpass filtering, unwanted artefacts appear to the filtered image. **What is this phenomenon called and why does it occur?**

`This phenomenon is called ringing. Ringing occurs because the image gets distorted or image loses high frequency information about the image.`

**4.6. Now, construct Gaussian lowpass and highpass filters with cut-off frequency `D0=0.2` and display them in the same figure.**

Hint: All you need to do is to modify the filter matrix `Hlp` line in the example code snippet accordingly to form `Hlpg` and `Hhpg` (see, formula 4.3-7 in the course book or the lecture notes, specifically, the **chapter04.pdf**).

In [ ]:
GausLP = np.exp(-D**2/2/D0**2)
GausHP = 1-GausLP
# display the filter masks
fig, ax = plt.subplots(1, 2)
ax[0].imshow(GausLP)
ax[0].set_title('Gaussian lowpass')
ax[1].imshow(GausHP)
ax[1].set_title('Gaussian highpass')
fig.tight_layout()

**4.7. Perform Gaussian lowpass and highpass filtering to the original test image and display the magnitude of the resulting FTs in the same figure.**

In [ ]:
# apply gaussian lowpass and highpass filtering to the test image
GLP = np.multiply(centered, GausLP)
GHP = np.multiply(centered, GausHP)
# display the magnitude of the resulting FTs
fig, ax = plt.subplots(1, 2)
ax[0].imshow(GLP.real)
ax[0].set_title('Gaussian lowpass')
ax[1].imshow(GHP.real)
ax[1].set_title('Gaussian highpass')
fig.tight_layout()

**4.8. Finally, reconstruct the filtered images like in step 4.5. and display the original image and the two Gaussian filtered images in the same figure.**

In [ ]:
# reconstruct the filtered images 
GLPF = (fftpack.ifft2(fftpack.ifftshift(GLP))).real
GHPF = (fftpack.ifft2(fftpack.ifftshift(GHP))).real
# display the three images in the same figure
fig, ax = plt.subplots(1, 3)
ax[0].imshow(test)
ax[0].set_title('original')
ax[1].imshow(GLPF)
ax[1].set_title('Gaussian lowpass')
ax[2].imshow(GHPF)
ax[2].set_title('Gaussian highpass')
fig.tight_layout()

**Do the unwanted artefacts appear in the Gaussian lowpass filtered image, why or why not?**

`The ringing does not appear in the Gaussian lowpass filtered image because the Gaussian filter is "non-negative and non-oscillatory".`

**What kind of effect does Gaussian (and ideal) lowpass filtering have on images in general? Why? What about highpass filtering? Why?**

`Gaussian lowpass filtering blurs the image, as does the ideal lowpass filter, and filters point out the smooth regions of the images. This happens because all of the values are positive and they all sum up to 1. Highpass filters points out the edges. Highpass filtering has positive and negative values and they all sump up to 0.`

# Aftermath
Finally, fill your answers to the following questions:

**How much time did you need to complete this exercise?**

`This exercise took me 2h 45 minutes.`

**Did you experience any problems with the exercise? Was there enough help available? Should this notebook be more (or less) detailed?**

`I had lot of problems with this exercise, almost on every task. I think this notebook should be more detailed as I could not figure out what to do and I could not find help from the internet or from the lectures.`

# References
`On 4.5: what causes ringing:
https://imaging.cs.msu.ru/en/research/ringing (11.4.2022)
On 4.8: no ringing in Gaussian lowpass filter:
https://en.wikipedia.org/wiki/Ringing_artifacts#Low-pass_filter (11.4.2022)
On 4.8: the effect of Gaussian (and ideal) lowpass(highpass filtering:
https://www.tutorialspoint.com/dip/high_pass_vs_low_pass_filters.htm (11.4.2022)`

# Submission

1. Before submitting your work, **check that your notebook (code) runs from scratch** and reproduces all the requested results by clicking on the menu `Kernel -> Restart & Run All`! Also, check that you have answered all the questions written in **bold**.
2. Clear all outputs and variables, etc. by click on the menu `Kernel -> Restart & Clear Output`. This may (or will) reduce the file size of your deliverable a lot! 
3. Rename this Jupyter notebook to **`DIP_PA3_[student number(s)].ipynb`** (e.g. `DIP_PA3_1234567.ipynb` if solo work or `DIP_PA3_1234567-7654321.ipynb` if pair work) and upload it as your submission to Moodle.